In [19]:
%run utils.ipynb 
%run models.ipynb

In [20]:
input_paras = {
    '#text-area-1' : "This is a paragraph that uses every single letter in the alphabet. Now, that does not mean this can be a paragraph with no story, but it does mean that every single letter is used. You can make it as generic or fanciful as you would like. You can talk about anything from quilts to jets to xylophones. Oh yeah, and you can use whatever language you want, from Afrikaans to Zulu.",
    '#text-area-2' : "One day, a zebra found a xylophone on the sidewalk. He quickly ran over, picked it up, and gave it to his pet mule. Just then, he found another xylophone. He kept that one for himself.",
    '#text-area-3' : "This is me trying to create a pangram for myself. A pangram is a sentence that contains all the characters from A-Z. Experimenting around such sentences might be useful for our project.",
    '#text-area-test' : "Well now that you are here already let me ask you this, do you think that the above paragraph is a pangram?"
}

sum([len(x) for x in input_paras.values()])

854

## Initialize 

- File paths and respective patterns to read set of all **input files**
- What kind of platforms to ignore, final structure of data, ignore users where data collected is less

In [41]:
path_pattern = {

    '../data/*' : r'dump\d+_\d+',
    '../../../exp/Data/*' : r'dump\d+_\d+'
}

known_invalid_platforms = ['Linux aarch64','Linux armv8l']

features_final = ['name','device_type','box_id','false_character','hold_for','key_pressed','long_pressed_equivalent','platform','pressed_after','type_combination','effort','speed']

user_count_threshold = 600

In [42]:
df = get_valid_dataset(path_pattern, known_invalid_platforms, user_count_threshold)
print_data_summary(df)
df = get_unique_users_subset(df)
df.shape, org_df.shape

******** Data at a high level ********
Number of users              :  24
Unique user_ids              :  26
Avg rows collected/user_id   :  966.88
Rows with false characters   : 34.84%
Avg long_pressed_equivalent  :  1.0
Rows with long_pressed > 1   : 0.0 %
*********************************************
*** 1 users with multiple user_ids, taking most significant instance for each user


((23632, 13), (29083, 13))

In [43]:
raw_clean = clean_raw_data(df)
raw_clean.shape

100%|██████████| 24/24 [00:00<00:00, 150.94it/s]

******** TRUE_CHAR_MEAN ********
Removing users with avg character perc for TRUE less than 60 (TRUE PERC THRESHOLD)
ALL USERS AVG    :  64.59553675799403
INVALID USERS AVG :  29.258773458488477
VALID USERS AVG   :  85.79759473769735
********************************
15 valid users found, data is cleaned for exploration and feature engineering!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


(14532, 13)

In [44]:
raw_clean.to_csv('../data/clean_union_04_28_20_20_00.csv', index=False)


In [45]:
df.shape

(23632, 13)

In [49]:
org_df = get_valid_dataset(path_pattern, known_invalid_platforms, user_count_threshold, 3)
print_data_summary(org_df)

raw_clean_all_rows_below_3 = clean_raw_data(org_df, 60, 3)
raw_clean_all_rows_below_3.shape

100%|██████████| 26/26 [00:00<00:00, 157.80it/s]

******** Data at a high level ********
Number of users              :  26
Unique user_ids              :  28
Avg rows collected/user_id   :  1038.68
Rows with false characters   : 32.67%
Avg long_pressed_equivalent  :  1.11
Rows with long_pressed > 1   : 10.51 %
*********************************************
******** TRUE_CHAR_MEAN ********
Removing users with avg character perc for TRUE less than 60 (TRUE PERC THRESHOLD)
ALL USERS AVG    :  67.24850028217601
INVALID USERS AVG :  29.685597724319198
VALID USERS AVG   :  87.13474281280608
********************************
17 valid users found, data is cleaned for exploration and feature engineering!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


(19367, 13)

In [50]:
raw_clean_all_rows_below_3['long_pressed_equivalent'].value_counts()

1    16927
2     2440
Name: long_pressed_equivalent, dtype: int64

In [51]:
len(raw_clean_all_rows_below_3['name'].unique())

17

In [52]:
raw_clean_all_rows_below_3.to_csv('../data/clean_union_exploration_04_29_20_22_00.csv', index=False)

## Prepare & Derive Features for Dataset

In [6]:
df = raw_clean

In [7]:
cond1 = df['key_pressed'].str.len() == 1
cond2 = df['long_pressed_equivalent'] == 1
conditions = cond1 & cond2
df = prepare_data(df, conditions)

********* DATA PREPARATION FOR FEATURE ENGINEERING *********
	RAW VS PREPARED SHAPES
(14532, 13)		(13378, 14)

----------
	CHARACTER TYPE DISTRIBUTION
          type
lower  54.167
upper  27.083
other  18.750
----------
	FALSE CHARACTER DISTRIBUTION
    false_character
f           87.375
t           12.625
*************************************************************


In [12]:
zipped_rows_2 = list(zip(df.values[:-1], df.values[1:]))
feature_df_2 = combine_characters_v1(zipped_rows_2, df.columns)

# """
zipped_rows_3 = list(zip(df.values[:-2], df.values[1:-1], df.values[2:]))
feature_df_3 = combine_characters_v1(zipped_rows_3, df.columns)
# """

100%|██████████| 13377/13377 [01:14<00:00, 179.88it/s]


****** Skipped 72 rows as `speed` evaluates to `inf`


100%|██████████| 13376/13376 [01:10<00:00, 189.19it/s]


****** Skipped 140 rows as `speed` evaluates to `inf`


In [13]:
len(feature_df_2.name.unique()), len(feature_df_2.user_id.unique())

(15, 15)

In [14]:
len(feature_df_3.name.unique()), len(feature_df_3.user_id.unique())

(15, 15)

In [15]:
feature_df_2.shape, feature_df_3.shape

((13143, 17), (12915, 17))

In [16]:
df_2 = feature_df_2[features_final]
df_3 = feature_df_3[features_final]

In [17]:
df_3

,name,device_type,box_id,false_character,hold_for,key_pressed,long_pressed_equivalent,platform,pressed_after,type_combination,effort,speed
0,Varun Sapre,Computer/Laptop,#text-area-1,fff,121,thi,3,Win32,181,lll,224,0.0967
0,Varun Sapre,Computer/Laptop,#text-area-1,fff,203,hi,3,Win32,230,llo,353,0.3091
0,Varun Sapre,Computer/Laptop,#text-area-1,fff,252,i,3,Win32,277,loo,506,0.2874
0,Varun Sapre,Computer/Laptop,#text-area-1,fff,324,p,3,Win32,269,ool,466,2.6667
0,Varun Sapre,Computer/Laptop,#text-area-1,fff,250,pr,3,Win32,311,oll,434,2.6726
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Tarun,Computer/Laptop,#text-area-test,fff,277,s p,3,Win32,272,lol,519,0.6907
0,Tarun,Computer/Laptop,#text-area-test,fff,252,pa,3,Win32,281,oll,430,0.4801
0,Tarun,Computer/Laptop,#text-area-test,fff,249,pan,3,Win32,218,lll,328,0.3548
0,Tarun,Computer/Laptop,#text-area-test,fff,219,ang,3,Win32,514,lll,694,0.1705


In [62]:
df_2.to_csv('../data/clean_union_fe_c2_04_28_20_21_00.csv', index=False)
df_3.to_csv('../data/clean_union_fe_c3_04_28_20_21_00.csv', index=False)

In [63]:
combine_df = df_2.append(df_3)
combine_df.shape

(26058, 12)

In [64]:
combine_df.to_csv('../data/clean_union_fe_c2_c3_04_28_20_21_00.csv', index=False)

In [16]:
df.head(2)

,name,device_type,box_id,false_character,hold_for,key_pressed,long_pressed_equivalent,platform,pressed_after,type_combination,effort,speed
0,Varun Sapre,Computer/Laptop,#text-area-1,ff,88,th,2,Win32,158,ll,168,0.1500
0,Varun Sapre,Computer/Laptop,#text-area-1,ff,97,hi,2,Win32,103,ll,120,0.0435
